In [13]:
# Task1_0620. 네이버 영화 순위 사이트에서 영화제목, 가격, 타입(구매 or 대여) 정보를 가져와서 TITLE, PRICE, TYPE 3개의 컬럼과 100개의 데이터포인트로 구성된 데이터프레임을 출력하세요.
import pandas as pd
from bs4 import BeautifulSoup
import requests
import chardet
import re
import urllib.request as rq

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
}

url = "https://serieson.naver.com/v3/movie/ranking/realtime"
response = requests.get(url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # 제목 가져오기
    titles = soup.find_all(class_='Title_title__s9o0D')
    title_texts = [title.get_text() for title in titles]

    print(len(title_texts))

    # 가격 가져오기
    prices = soup.find_all(class_='Price_price__GqXqo')
    price_texts = [price.get_text() for price in prices]
    print(len(price_texts))

    # 타입 가져오기
    types = soup.find_all(class_='Price_text__pRk_f')
    type_texts = [type.get_text() for type in types]
    print(len(type_texts))

min_length = min(len(title_texts), len(price_texts), len(type_texts))
title_texts = title_texts[:min_length]
price_texts = price_texts[:min_length]
type_texts = type_texts[:min_length]


data = {
    'Title':title_texts,
    'Prcie':price_texts,
    'Type':type_texts
}
df = pd.DataFrame(data)

df.to_csv('movie.csv', index=False, encoding='utf-8-sig')
df

86
87
86


,Title,Prcie,Type
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),"7,150캐시",구매
1,콩나물,"1,000캐시",구매
2,태극기 휘날리며 - 특별판,"10,000캐시",구매
3,설계자,"11,000캐시",대여
4,원피스 필름 레드,"1,540캐시",대여
...,...,...,...
81,마스크,"4,950캐시",대여
82,사운드 오브 뮤직,"1,300캐시",구매
83,마지막 한 걸음까지,"5,500캐시",구매
84,흑인 오르페,500캐시,구매


In [14]:
# teach
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 설정
url = 'https://serieson.naver.com/v3/movie/ranking/realtime'

# HTTP GET 요청
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 영화 제목과 가격을 저장할 리스트
titles = []
prices = []
types = []

# 영화 제목과 가격 추출
movies = soup.select('.RankingList_ranking_list__N4QsH > li')

for movie in movies:
    title_tag = movie.select_one('.Title_title__s9o0D')
    if title_tag:
        title = title_tag.get_text(strip=True)
    
        # 가격 및 타입 추출
        price_items = movie.select('.Price_price_item__tOOqb')
        for price_item in price_items:
            type_tag = price_item.select_one('.Price_text__pRk_f')
            price_tag = price_item.select_one('.Price_price__GqXqo')
            if type_tag and price_tag:
                type_text = type_tag.get_text(strip=True)
                price = price_tag.get_text(strip=True)
                
                # 가격에서 "캐시" 및 SVG 태그 제거
                price = re.sub(r'캐시.*', '', price).strip()
                
                titles.append(title)
                prices.append(price)
                types.append(type_text)

# 데이터프레임 생성
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Type': types
})

# 데이터프레임 출력
df

,Title,Price,Type
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),"7,150",구매
1,콩나물,"1,000",구매
2,태극기 휘날리며 - 특별판,"10,000",구매
3,설계자,"11,000",대여
4,원피스 필름 레드,"1,540",대여
...,...,...,...
81,마스크,"1,300",대여
82,사운드 오브 뮤직,"5,500",구매
83,마지막 한 걸음까지,500,구매
84,흑인 오르페,500,구매


In [50]:
import requests as rq
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}
url = 'https://news.daum.net/politics#1'
r = rq.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('.link_txt')
body = '\n'.join([f'{i+1}. {line.text}' for i, line in enumerate(lines)])
print(body)

1. 22대 국회서 재시동건 충청 현안들…정치력 신장 없인 전망 어둡다
2. 부부 동시 임관 장교 탄생…“태권도학과 함께 졸업 혼인신고 후 한길 걷기로”
3. 국민의힘 박정하 의원, 강원특별자치도당위원장 연임
4. 與 원희룡·한동훈·원희룡·나경원 3파전?...이준석 "尹이 간섭할 것" [앵커리포트]
5. 與 "30년묶인 상속·증여 공제 한도 인상 필요"…감세 정책 시동
6. 민주 "법사·운영위 1년씩 교대, 비현실적"…간호법 당론 채택도
7. 여야 수석, 원구성 협상 또 결렬…"주말도 만나 계속"
8. 목포·여수·순천·광양·해남, 기회발전특구 지정
9. 김정은의 ‘풍산개 외교’…文 이어 푸틴에게도[청계천 옆 사진관]
10. 여야 원내수석, 원구성 또 '빈손 회동'…원내대표는 안 만나고 '으르렁'
11. 여야, 연일 ‘빈손 회동’…원 구성 협상 난항
12. [북러 회담] "러, 조약 근거로 북한과 무기거래 공식화할듯"
13. 한반도 상공 등장한 '하늘의 전함' AC-130J [양낙규의 Defence Club]
14. 춘천시 정원도시 협력지원단 '첫발'…8개 기관·단체 구성
15. [성공예감] 지역이 살아야 출생률도 올라간다, 해법은 – 양승훈 교수(경남대 사회학과), 서영민 기자(KBS)
16. 민주당 전남도의회 의장후보에 김태균 선출…27일 본회의서 투표
17. 한동훈 23일 당대표 출마 선언…이재명 연임 도전 수순
18. 野, 채상병특검 청문회 불출석 증인 ‘무더기 고발’ 예고…“신원식·김계환만 사유서 제출”
19. 나경원 "근본 깨는 이재명, 대한민국 비명횡사..무조건 막아야, 곧 결심"
20. 박원석 "원희룡 출마? 용산 참전신호"...김근식 "결선투표 노린 듯"
21. 나경원 “당 대표는 대통령과의 갈등이 겉으로 드러나면 안 돼…차별화해 대권 가겠다는 건 미래 없어”
22. 최진녕 변호사 / 서용주 전 더불어민주당 상근부대변인 - 원 구성 놓고 여야 입장 차…대치 정국 심화
23. ③중산층까지 악영향? 금투세 쟁점과 필요한 결정은?
24. 위계충 권폭귀

In [18]:
# Task2_0620. url = 'https://news.daum.net/politics#1'은 정치기사 1페이지인데 1~10페이지에 있는 기사를 모두 출력하세요. 
import requests as rq
from bs4 import BeautifulSoup
import re

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}
url = 'https://news.daum.net/politics#1'
r = rq.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('.link_txt')
body = '\n'.join([f'{i+1}. {line.text}' for i, line in enumerate(lines)])
print(body)

1. 북러 고강도 군사 밀착에 한미도 면밀 대응…외교장관 긴급 통화
2. [정치쇼] 김재섭 "천하제일 '비윤'호소대회…원희룡·尹 모종의 일 있었을 것"
3. “영부인께 로비하고 싶습니다”… 권익위 ‘조롱’ 잇따라
4. 북·러 ‘군사 협력’ 범위 불분명…“생각보다 더 장기적·더 위험”
5. 김현기 서울시의장, 몽골 울란바토르에 소방차 10대 전달
6. 북한, '6·25 미제 반대 투쟁의 날' 앞두고…"원수 격멸 의지 굳건"
7. 박찬대 "건희위원회로 전락한 권익위"…고무줄 청탁금지법 지적
8. 대통령실 '야 단독소집' 운영위 불참키로…야, 증인 채택할듯
9. "우크라 무기지원은 러 행동에 달려"…155㎜ 포탄 등 'K-방산' 대기중
10. [사실은 이렇습니다] 과기정통부 “출연연 지원·육성 관리 방안 6월 말 발표 예정”
11. 박찬대 "尹, 해병대원 특검법 또다시 거부한다면 범죄자라 자백한 것으로 여길 것"
12. KFN, 북한도발사 다큐 시즌2 방영…"38선 국지전, DMZ 도발 등 조명"
13. KF-21 ‘원거리 작전’ 공중급유 성공… 비행시험 3분의 1 순항
14. 한-폴, FA-50 항공정비 협력합의…"수출 후 A/S도 확실한 K-방산"
15. 보령시, 해수욕장 개장 대비 원산지 표시 특별점검
16. ‘성능 + 가격 + 확장성’ KF-21… 전투기서도 K방산 성공신화 잇는다
17. 김재섭 "천하제일 '비윤' 호소대회…원희룡·尹 모종의 일 있었을 것"
18. 한미동맹엔 없지만 북중동맹에 있는 것…새로운 북러 조약에도?
19. ‘수구 역주행’ 보수정당, 검찰국가 파트너로 전락
20. 퀸익위
21. 철문 걸어잠근 기재부 세제실, ‘철통보안’ 모드 된 까닭은
22. 이재명과 한동훈의 급소
23. 이철희, “한동훈 당대표 될 듯, 남 공격 그만하고 자기 얘기 해야”
24. 영일만 석유개발과 한국의 꿈
25. 정부는 오답을 쓰고 있다
26. 뉴스홈
27. 사회
28. 정치
29. 경제
30. 국제
31. 문화
32. IT
33. 포토
34. 제휴 언론사
35. 

In [30]:
#task2
import requests
from bs4 import BeautifulSoup
import re

for j in range(1, 11):
    #https://news.daum.net/breakingnews/politics?page=1
    url = f'https://news.daum.net/breakingnews/politics?page={j}'
    print(url)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    lines = soup.find_all('a',class_="link_txt")
    lines = lines[7:-21] # 기사제목만 추출할 수 있게 광고 및 헤드라인 불필요. 8자 이하 문자열은 필터하기 위한 슬라이싱.
    for i, tag in enumerate(lines, 1):
        text = tag.get_text().strip().replace(',', '')
        matches = re.findall('[가-힣0-9]+', text)
        print(f"{i}. {' '.join(matches)}")
    print(f'페이지수: {j}')

https://news.daum.net/breakingnews/politics?page=1
1. 보훈부 보훈위탁병원 비상진료체계 현장 점검
2. 제73회 서울시문화상 추천 접수
3. 발언하는 박찬대 원내대표
4. 탈북민단체 대북전단 30만장 살포
5. 연금 전문가들 한국 연금체계 지속 불가능
6. 미 어떤 지원도 환영 미군 대응 태세 평가
7. 생각에 잠긴 이재명 대표
8. 발언하는 이재명 대표
9. 발언하는 이재명 대표
10. 게시판 한화에어로 6 25 참전용사 맞춤형 신발 헌정
11. 더불어민주당 최고위원회의
12. 생각에 잠긴 이재명 대표
13. 윤상현 일주일 전 나 돕겠다던 원희룡 어제는 죄송하다고
14. 더불어민주당 최고위원회의
15. 최고위 발언하는 이재명 대표
페이지수: 1
https://news.daum.net/breakingnews/politics?page=2
1. 최고위 발언하는 이재명 대표
2. 발언하는 이재명 대표
3. 민주당 최고위원회의
4. 발언하는 이재명 대표
5. 건희권익위로 개명하라 박찬대 김 여사 면죄부 주려고 청탁금지법 왜곡 해석
6. 발언하는 이재명 대표
7. 최고위 발언하는 이재명 대표
8. 최고위 발언하는 이재명 대표
9. 최고위 발언하는 이재명 대표
10. 한동훈의 러닝메이트 누구 친윤계는 당헌당규 셀프 개정 가능성으로 압박 이런정치
11. 김성태 나경원은 온실 속 민들레 들꽃 되기도 받기도 어정쩡
12. 포토 최고위 발언하는 이재명
13. 속보 넘은 북한군 여러명 우리 군 경고사격으로 북상
14. 원희룡 모레 국회서 국민의힘 대표 출마 선언 한동훈과 한날 출사표
15. 최고위 참석하는 이재명 대표
페이지수: 2
https://news.daum.net/breakingnews/politics?page=3
1. 최고위회의 참석하는 이재명 대표
2. 최고위회의 참석하는 이재명 대표
3. 생각에 잠긴 이재명 대표
4. 생각에 잠긴 이재명 대표
5. 탈북단체 대북전단 30만 장 살포 오물풍선으로 맞대응 가능성
6. 외교부 주한 

In [19]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# 기본 URL 설정 (여기에 페이지 번호를 추가하여 여러 페이지를 요청)
base_url = 'https://news.daum.net/politics#'  # 실제 URL 형식으로 대체해야 합니다
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
}

# 데이터 저장용 리스트 초기화
all_data = []

# 페이지 범위 설정 (예: 1부터 10까지)
for page_num in range(1, 11):
    url = f"{base_url}{page_num}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')

        for tag in lines:
            text = tag.get_text().strip().replace(',', '')
            matches = re.findall('[가-힣0-9]+', text)
            cleaned_text = ' '.join(matches)
            all_data.append([page_num, cleaned_text])
    else:
        print(f"HTTP 요청 실패: {response.status_code} (페이지 {page_num})")


df = pd.DataFrame(all_data, columns=['Page', 'Text'])
df
# page_10 = df[df['Page'] == 10]
# print(page_10)

,Page,Text
0,1,북러 고강도 군사 밀착에 한미도 면밀 대응 외교장관 긴급 통화
1,1,정치쇼 김재섭 천하제일 비윤 호소대회 원희룡 모종의 일 있었을 것
2,1,영부인께 로비하고 싶습니다 권익위 조롱 잇따라
3,1,북 러 군사 협력 범위 불분명 생각보다 더 장기적 더 위험
4,1,김현기 서울시의장 몽골 울란바토르에 소방차 10대 전달
...,...,...
145,10,철문 걸어잠근 기재부 세제실 철통보안 모드 된 까닭은
146,10,이재명과 한동훈의 급소
147,10,이철희 한동훈 당대표 될 듯 남 공격 그만하고 자기 얘기 해야
148,10,영일만 석유개발과 한국의 꿈


In [45]:
# 한글 과 숫자만 출력, 콤마 제거

soup = BeautifulSoup(html, 'html.parser')
lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')

data = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    cleaned_text = ' '.join(matches)
    data.append([i, cleaned_text])


df = pd.DataFrame(data, columns=['Index', 'Text'])
df

,Index,Text
0,1,이준석 어물전 엎어놓을 사람 어대한 나경원 선전할 것
1,2,조태용 국정원장이 민주당 의원과 오찬을 갑자기 취소한 이유는
2,3,노컷브이 법사 운영위 1년씩 대통령도 1년씩 원구성 난항
3,4,이준석 전당대회 윤 대통령 의중 작용 시작 한동훈 포기 압박
4,5,광주 남구소식 등산로 고장 운동기구 점검 보수
5,6,의협 올바른 의료를 위한 특별위원회 출범 전공의 자리 공석
6,7,채상병특검 청문회 불출석 증인 무더기 고발 예고 신원식 김계환만 사유서 제출
7,8,나경원 근본 깨는 이재명 대한민국 비명횡사 무조건 막아야 곧 결심
8,9,박원석 원희룡 출마 용산 참전신호 김근식 결선투표 노린 듯
9,10,나경원 당 대표는 대통령과의 갈등이 겉으로 드러나면 안 돼 차별화해 대권 가겠다는 ...


In [24]:
# Task3_0620. 앞에서 출력한 기사 리스트를 pandas 데이터프레임으로 변환 후 csv 파일로 저장 후 다시 불러오세요.
soup = BeautifulSoup(html, 'html.parser')
lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')

data = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    cleaned_text = ' '.join(matches)
    data.append([i, cleaned_text])


df = pd.DataFrame(data, columns=['Index', 'Text'])
df

df.to_csv('csv_file.csv', index=False, encoding='utf-8-sig')
df = pd.read_csv('csv_file.csv')
df

,Index,Text
0,1,국민의힘 유승민 당대표 불출마 선언 어대한 에 힘 실릴까
1,2,원희룡 출마 발표 전날 용산 찾아 대통령 존중한다
2,3,정치쇼 김재섭 천하제일 비윤 호소대회 원희룡 모종의 일 있었을 것
3,4,북러 고강도 군사 밀착에 한미도 면밀 대응 외교장관 긴급 통화
4,5,영부인께 로비하고 싶습니다 권익위 조롱 잇따라
5,6,북 러 군사 협력 범위 불분명 생각보다 더 장기적 더 위험
6,7,전대 불붙는데 어대명 민주당 흥행카드가 없다
7,8,이광수 광수네복덕방 대표 정치권 논쟁 중 종합부동산세 폐지 존치
8,9,김재섭 천하제일 비윤 호소대회 원희룡 모종의 일 있었을 것
9,10,한미동맹엔 없지만 북중동맹에 있는 것 새로운 북러 조약에도


In [26]:
import re
import pandas as pd

soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div.main-sub > div > ul > li > strong > a')
results = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    result = ' '.join(matches)
    results.append({'Index':i, 'Text':result})
    
df = pd.DataFrame(results)

df.to_csv('news.csv', index=False)
df = pd.read_csv('news.csv')
df

,Index,Text
0,1,김현기 서울시의장 몽골 울란바토르에 소방차 10대 전달
1,2,북한 6 25 미제 반대 투쟁의 날 앞두고 원수 격멸 의지 굳건
2,3,박찬대 건희위원회로 전락한 권익위 고무줄 청탁금지법 지적
3,4,대통령실 야 단독소집 운영위 불참키로 야 증인 채택할듯
4,5,사실은 이렇습니다 과기정통부 출연연 지원 육성 관리 방안 6월 말 발표 예정
5,6,박찬대 해병대원 특검법 또다시 거부한다면 범죄자라 자백한 것으로 여길 것
6,7,북한도발사 다큐 시즌2 방영 38선 국지전 도발 등 조명
7,8,21 원거리 작전 공중급유 성공 비행시험 3분의 1 순항
8,9,한 폴 50 항공정비 협력합의 수출 후 도 확실한 방산
9,10,보령시 해수욕장 개장 대비 원산지 표시 특별점검
